# Spider Fly testing environment

In [1]:
from custom_agents.CTCE_algorithms.ma_sac_agents_seq_discrete import Agents
from custom_spider_env.spider_fly_env.envs.grid_MA_pettingzoo import SpiderFlyEnvMA
from custom_spider_env.spider_fly_env.envs.pettingzoo_wrapper import PettingZooWrapper

env = SpiderFlyEnvMA(size = 3, spiders = 2, max_timesteps = 100)
env = PettingZooWrapper(env, normalize = True)

agents = Agents(env, batch_size = 256, layer_sizes = (32, 32), global_observations = True) 
# agents = Agents(env, batch_size = 256, global_observations = True) 

2024-05-20 11:53:00.355902: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/hkolstee/.local/lib/python3.10/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment SpiderFlyGrid-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/hkolstee/.local/lib/python3.10/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment SpiderFlyGridMA-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [ ]:
rewards = agents.train(nr_steps = 30000)

KeyboardInterrupt: 

In [1]:
from custom_agents.CTCE_algorithms.ma_sac_agents_seq_discrete import Agents
from custom_spider_env.spider_fly_env.envs.grid_MA_pettingzoo2 import SpiderFlyEnvMA
from custom_spider_env.spider_fly_env.envs.pettingzoo_wrapper import PettingZooWrapper

env = SpiderFlyEnvMA(spiders=3, size=4, max_timesteps = 100)
env = PettingZooWrapper(env, normalize = True)

agents = Agents(env, batch_size = 256, layer_sizes = (128, 128), global_observations = True, lr_actor = 0.00003, lr_critic = 0.00003) 
# agents = Agents(env, batch_size = 256, global_observations = True) 

2024-05-20 13:29:29.491954: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/hkolstee/.local/lib/python3.10/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment SpiderFlyGrid-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/hkolstee/.local/lib/python3.10/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment SpiderFlyGridMA-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [2]:
rewards = agents.train(nr_steps = 20000 * 100, warmup_steps = 10000)

KeyboardInterrupt: 

In [ ]:
from custom_agents.CTCE_algorithms.ma_sac_agents_seq_discrete import Agents
from custom_spider_env.spider_fly_env.envs.grid_MA_pettingzoo import SpiderFlyEnvMA
from custom_spider_env.spider_fly_env.envs.pettingzoo_wrapper import PettingZooWrapper

import numpy as np

import pandas as pd

env = SpiderFlyEnvMA(spiders=3, size=4, max_timesteps = 100)
env = PettingZooWrapper(env)

agents = Agents(env, batch_size = 8, layer_sizes = (256, 256), global_observations = True) 

agents.train(nr_steps = 150000, warmup_steps = 20000)

/home/hkolstee/.local/lib/python3.10/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment SpiderFlyGrid-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/hkolstee/.local/lib/python3.10/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment SpiderFlyGridMA-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/hkolstee/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return 

KeyboardInterrupt: 

In [ ]:
agents.actor.save("models/SAC_hard3", "actor")
agents.critic1.save("models/SAC_hard3", "critic1")
agents.critic2.save("models/SAC_hard3", "critic2")
agents.critic1_targ.save("models/SAC_hard3", "critic1_targ")
agents.critic1_targ.save("models/SAC_hard3", "critic2_targ")

# agents.actor.save_checkpoint("models/SAC_hard3", "actor", loss = 0, step = 1000*100)
# agents.critic1.save_checkpoint("models/SAC_hard3", "critic1", loss = 0, step = 1000*100)
# agents.critic2.save_checkpoint("models/SAC_hard3", "critic2", loss = 0, step = 1000*100)
# agents.critic1_targ.save_checkpoint("models/SAC_hard3", "critic1_targ", loss = 0, step = 1000*100)
# agents.critic1_targ.save_checkpoint("models/SAC_hard3", "critic2_targ", loss = 0, step = 1000*100)


# TEST of custom Multi-Agent SAC agent on citylearn env

In [ ]:
import gymnasium as gym
import seaborn as sns
import matplotlib.pyplot as plt

import torch

import numpy as np
import pandas as pd

from citylearn.citylearn import CityLearnEnv
from citylearn.wrappers import NormalizedSpaceWrapper, StableBaselines3Wrapper

from custom_agent.CTDE.ma_sac_agents_seq import Agents

from custom_reward.custom_reward import CustomReward

In [ ]:
schema_path = "data/schema.json"

env = CityLearnEnv(schema = schema_path, reward_function = CustomReward, central_agent=False)

# wrap environment for a more workable env
env = NormalizedSpaceWrapper(env)

In [ ]:
sac_agent = Agents(env, batch_size=100, buffer_max_size=100000)

In [ ]:
env.observation_space

In [ ]:
env.action_space

In [ ]:
warmup_steps = 5000

# make agent
sac_agent = Agents(env, batch_size=256, buffer_max_size=100000)

# training run
sac_agent.train(nr_steps = 5500 * 720, warmup_steps = warmup_steps, learn_delay = 100, learn_freq = 1, learn_weight = 1, save_dir = "temp_models")
